## Segmentacion de Imagenes con CamVid
Traducción al español del la [clase de Fast.ai](https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson3-camvid-tiramisu.ipynb) por [Fernando Bernuy B.](https://scholar.google.cl/citations?user=Q4tEQYYAAAAJ&hl)

Partiremos inicializando el notebook y las librerias

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#import torch
#torch.cuda.set_device(2)
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

## Cargar Base de Datos
Para este caso utilizaremos una versión modificada de CamVid del paper `The One Hundred Layer Tiramisu`, con imágenes pequeñas y menos clases. Esta versión de CamVid esta disponible en el siguiente repositorio:

    git clone https://github.com/alexgkendall/SegNet-Tutorial.git

Al igual que en el caso anterior, vamos a usar links simbólicos de los datos de la base de datos para evitar problemas con la descarga, pero en condiciones normales, deberíamos hacer un `clone` del repositorio.

In [ ]:
!mkdir ~/SegNet-Tutorial
!mkdir ~/SegNet-Tutorial/CamVid
!ln -s /data/home/instructor1/SegNet-Tutorial/CamVid/t* ~/SegNet-Tutorial/CamVid
!ln -s /data/home/instructor1/SegNet-Tutorial/CamVid/v* ~/SegNet-Tutorial/CamVid

In [ ]:
#!cd ~
#!git clone https://github.com/alexgkendall/SegNet-Tutorial.git

Creamos el objeto Path apuntando a la base de la carpeta con la base de datos que descargamos y miramos el contenido.

In [ ]:
path = Path.home()/'SegNet-Tutorial/CamVid'

In [ ]:
path.ls()

## Data

¿Cómo están estructuradas las imágenes y sus etiquetas?

In [ ]:
fnames = get_image_files(path/'val')
fnames[:3]

In [ ]:
lbl_names = get_image_files(path/'valannot')
lbl_names[:3]

Creamos un método que retorna el path a la imagen con las etiquetas a partir del path de la imagen original. Además, incluimos la equivalencia de etiqueta a `str` para nuestra interpretación.

In [ ]:
def get_y_fn(x): return Path(str(x.parent)+'annot')/x.name

codes = array(['Sky', 'Building', 'Pole', 'Road', 'Sidewalk', 'Tree',
    'Sign', 'Fence', 'Car', 'Pedestrian', 'Cyclist', 'Void'])

Ahora veamos una imagen y su etiquetado:

In [ ]:
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(5,5))

mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)

In [ ]:
src_size = np.array(mask.shape[1:])
src_size,mask.data

## Generamos los Datasets

Utilizaremos la mitad del tamaño de la imagen y debemos configurar el `batch_size` para que pueda funcionar en nuestro equipo.

In [ ]:
bs,size = 6,src_size//2

In [ ]:
src = (SegmentationItemList.from_folder(path)
       .split_by_folder(valid='val')
       .label_from_func(get_y_fn, classes=codes))

In [ ]:
data = (src.transform(get_transforms(), tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [ ]:
data.show_batch(2, figsize=(10,7))

## Cargamos el Modelo

La base de datos contiene una etiqueta `void` para referirse a todos los elementos que no estan incluidos dentro de las otras etiquetas, y no deben ser consideradas para la evaluacion del desempeño, por lo que debemos implementar una funcion de `accuracy` que considere este efecto como métrica.

In [ ]:
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

Para la segmentación utilizaremos la estructura [UNET](https://docs.fast.ai/vision.models.unet.html#DynamicUnet) a partir de un `resnet34`. Para esto utilizaremos la función [`unet_learner`](https://docs.fast.ai/vision.learner.html#unet_learner), que nos entregarar el `learner`apropiado para esta tarea.

In [ ]:
metrics=acc_camvid
wd=1e-2

In [ ]:
learn = unet_learner(data, models.resnet34, metrics=metrics, wd=wd, bottle=True).to_fp16()

elegimos un buen LR

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=1e-4

y entrenamos un ciclo

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.8)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1');

Ahora hacemos fine-tunning de la red

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr/100,lr)

In [ ]:
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.save('stage-2');

veamos como se comporto el entrenamiento

In [ ]:
learn.recorder.plot()
learn.recorder.plot_lr()
learn.recorder.plot_losses()

## Ahora en grande

Vamos a utilizar las imágenes en tamaño completo. Para esto tenemos que corregir el tamaño en el `transform`, poner un `batch_size` apropiado y generar un nuevo `learn`.

Lo primero que haremos será borrar el `learn` anterior, para liberar la memoria de la GPU, y luego crearemos el nuevo.

In [ ]:
learn=None
gc.collect()

In [ ]:
size = src_size
bs=4

In [ ]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [ ]:
learn = unet_learner(data, models.resnet34, metrics=metrics, wd=wd, bottle=True).load('stage-2').to_fp16();

buscamos un LR apropiado

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=1e-4

entrenamos un ciclo

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.8)

In [ ]:
learn.save('stage-1-big')

observamos el comportamiento del entrenamiento

In [ ]:
learn.recorder.plot()
learn.recorder.plot_lr()
learn.recorder.plot_losses()

In [ ]:
learn.load('stage-1-big');

Hacemos una última etapa de fine-tunning

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr/1000,lr/10)

In [ ]:
learn.fit_one_cycle(10, lrs)

In [ ]:
learn.save('stage-2-big')

¿Como son los resultados?

Nos compararemos con el paper de [DenseNet](https://arxiv.org/pdf/1611.09326.pdf) (0.915)

In [ ]:
learn.load('stage-2-big');

In [ ]:
learn.show_results(rows=3, figsize=(9,11))

In [ ]:
# start: 480x360

In [ ]:
print(learn.summary())

In [ ]:
learn.destroy()

In [ ]:
! rm -r .fastai/data

## FIN